In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0, '../src')

from pathlib import Path
import pandas as pd
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from pns_dict import PNS_DICT_RAW
import numpy as np
from preprocessing_constants import SELECTED_COLUMNS, UPDATE_DICT

PNS_DATA = Path('../data/raw/PNS_2019/PNS_2019.zip').resolve()
zip_pense = ZipFile(PNS_DATA)

path_unzip = zip_pense.namelist()[0]
df = pd.read_csv(zip_pense.open(path_unzip), delimiter=';', low_memory=False)

In [2]:
print("O dataset PNS possui {} linhas e {} colunas (features).".format(*df.shape))

O dataset PNS possui 293726 linhas e 1087 colunas (features).


### Criando indicadores

Neste projeto usaremos o [PHQ9](https://www.mdcalc.com/calc/1725/phq9-patient-health-questionnaire9) para ter um indicador do grau de depressão da pessoa no momento da pesquisa. 

Além deste indicador, utilizaremos a questão Q092 (Algum médico ou profissional de saúde mental (como psiquiatra ou psicólogo) já lhe deu o diagnóstico de depressão?) como referência de filtragem de dados devido à questões mais específicas como 'Por causa da depressão Toma medicamentos' serem dependentes dela.

In [2]:
df.loc[(18 <= df['C008']) & (df['C008'] <= 59), 'phq9_total'] = df.loc[(18 <= df['C008']) & (df['C008'] <= 59), ['N010', 'N011', 'N012', 'N013', 'N014', 'N015', 'N016', 'N017', 'N018']].sum(axis=1, min_count=1) - 9 # -9 correção do valor, como o questionario a resposta "Nenhuma" começa no 1

# Classificando a gravidade da depressão com base na pontuação total
def classify_depression(row):
    value = row['phq9_total']

    if  0 <= value < 5:
        return 1
    elif value < 10:
        return 2
    elif value < 15:
        return 3
    elif value < 20:
        return 4
    elif value >= 20:
        return 5

    return np.nan # questionario não respondido ou idade não compativel

df['depression_severity'] = df.apply(classify_depression, axis=1)

: 

: 

# Modificações em variáveis de interesse

In [1]:
conditions = [
    df['D00901'].isin(['Creche', 'Pré-escola', 'Classe de alfabetização – CA',
                             'Alfabetização de jovens e adultos',
                             'Antigo primário (elementar)',
                             'Regular do ensino fundamental ou do 1º grau']),
    df['D00901'].isin(['Antigo ginasial (médio 1º ciclo)',
                             'Educação de jovens e adultos (EJA) ou supletivo do ensino fundamental',
                             'Regular do ensino médio ou do 2º grau']),
    df['D00901'].isin(['Antigo científico, clássico etc. (médio 2º ciclo)',
                             'Educação de jovens e adultos (EJA) ou supletivo do ensino médio',
                             'Superior – graduação']),
    df['D00901'].isin(['Especialização de nível superior (duração mínima de 360 horas)',
                             'Mestrado', 'Doutorado'])
]
new_classifications = [
    'Sem instrução e fundamental incompleto',
    'Fundamental completo e médio incompleto',
    'Médio completo e superior incompleto',
    'Superior completo'
]

# Agrega as escolaridades em um novo conjunto de classificações para facilitar analise
df['escolaridade_agregada'] = np.select(conditions, new_classifications, default=None)
a

NameError: name 'df' is not defined

In [ ]:
df['renda_trabalho'] = df['E01602']+df['E01604']+df['E01802']+df['E01804']
df['horas_semanais_trabalho'] = df['E017']+df['E019']

In [14]:
selected = df[df['depression_severity'].notna()] # pega somente os registros que foram respondidos como sim ou não
print("A quantidade de dados investigável dos dados são {}, {:02f}% do total.".format(selected.shape[0], 100*selected.shape[0]/df.shape[0]))

A quantidade de dados investigável dos dados são 65803, 22.402852% do total.


In [15]:
selected = df[(df['depression_severity'].notna()) | df['Q092'].isin([1, 2])] # pega somente os registros que foram respondidos como sim ou não ou que possui informações sobre dignostico de depressao
print("A quantidade de dados investigável dos dados são {}, {:02f}% do total.".format(selected.shape[0], 100*selected.shape[0]/df.shape[0]))

A quantidade de dados investigável dos dados são 90846, 30.928825% do total.


In [16]:
print("Numero de casas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS']).size().reset_index().shape[0])
print("Numero de pessoas entrevistadas: ", selected.groupby(['V0001', 'V0024', 'UPA_PNS', 'V0006_PNS', 'C00301']).size().reset_index().shape[0])

Numero de casas entrevistadas:  90846
Numero de pessoas entrevistadas:  90846


Com isso temos que as pessoas entrevistadas moram em casas diferentes, isso facilita a divisão dos dados.

#### Filtragem e renomeação das colunas de interesse

In [17]:
{k: PNS_DICT_RAW.get_question(k) for k in SELECTED_COLUMNS.keys()}

{'V0001': 'Unidade da Federação',
 'V0022': 'Total de moradores',
 'V0026': 'Tipo de situação censitária',
 'V0031': 'Tipo de área',
 'V0025A': 'Seleção do morador de 15 anos ou mais para responder o questionário individual',
 'VDF004': 'Faixa de rendimento domiciliar per capita (exclusive o rendimento das pessoas cuja condição na unidade domiciliar era pensionista, empregado doméstico ou parente do empregado doméstico)',
 'A001': 'Tipo do domicílio',
 'A002010': 'Qual é o material que predomina na construção das paredes externas deste domicílio?',
 'A003010': 'Material predominante na cobertura (telhado) do domicílio',
 'A004010': 'Qual é o material que predomina no piso deste domicílio',
 'A01001': 'Quantos cômodos têm este domicílio',
 'A011': 'Quantos cômodos estão servindo permanentemente de dormitório para os moradores deste domicílio',
 'A005010': 'Qual é a principal forma de abastecimento de água deste domicílio',
 'A005012': 'Este domicílio está ligado à rede geral de distribu

In [18]:
df_all = selected[SELECTED_COLUMNS.keys()]
df_all = df_all.rename(columns=SELECTED_COLUMNS)
df_all

,uf,moradores,tipo_zona,tipo_area,V0025A,renda_pc,casa_tipo,A002010,A003010,A004010,...,V034,V03501,V03502,V03503,Y001,Y00101,Y002,Y003,phq9_total,depression_severity
0,11,6.0,1,1,1.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,17.0,1.0,3.0,6.0,2.0
9,11,4.0,1,1,1.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,2.0,1.0
10,11,8.0,1,1,1.0,3.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,17.0,1.0,1.0,3.0,1.0
18,11,1.0,1,1,1.0,2.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,0.0,1.0
19,11,2.0,1,1,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293709,53,3.0,2,1,1.0,4.0,1.0,1.0,1.0,3.0,...,NaN,NaN,NaN,NaN,1.0,18.0,1.0,3.0,4.0,1.0
293712,53,2.0,2,1,1.0,4.0,1.0,1.0,1.0,3.0,...,1.0,2.0,1.0,NaN,4.0,NaN,1.0,3.0,1.0,1.0
293713,53,4.0,2,1,1.0,2.0,1.0,2.0,1.0,3.0,...,NaN,NaN,NaN,NaN,1.0,15.0,1.0,3.0,1.0,1.0
293718,53,3.0,2,1,1.0,5.0,1.0,1.0,1.0,1.0,...,2.0,NaN,1.0,NaN,1.0,16.0,1.0,3.0,4.0,1.0


In [19]:
missing_values = df_all.isna().sum()
missing_values

uf                         0
moradores                  0
tipo_zona                  0
tipo_area                  0
V0025A                     0
                       ...  
Y00101                 21515
Y002                    3939
Y003                   28623
phq9_total             25043
depression_severity    25043
Length: 379, dtype: int64

In [20]:
df_all[df_all['escolaridade'].isna()]

,uf,moradores,tipo_zona,tipo_area,V0025A,renda_pc,casa_tipo,A002010,A003010,A004010,...,V034,V03501,V03502,V03503,Y001,Y00101,Y002,Y003,phq9_total,depression_severity
19,11,2.0,1,1,1.0,1.0,2.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,4.0,NaN,3.0,NaN,0.0,1.0
44,11,4.0,1,1,1.0,7.0,1.0,1.0,1.0,1.0,...,2.0,NaN,2.0,NaN,1.0,15.0,1.0,2.0,6.0,2.0
57,11,1.0,1,1,1.0,4.0,2.0,1.0,2.0,1.0,...,2.0,1.0,2.0,NaN,1.0,15.0,1.0,2.0,0.0,1.0
65,11,3.0,1,1,1.0,5.0,2.0,1.0,2.0,1.0,...,NaN,NaN,NaN,NaN,1.0,16.0,1.0,1.0,0.0,1.0
76,11,3.0,1,1,1.0,3.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,3.0,NaN,3.0,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293565,53,4.0,2,1,1.0,2.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,1.0
293572,53,1.0,2,1,1.0,4.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,1.0,17.0,2.0,NaN,NaN,NaN
293589,53,3.0,2,1,1.0,3.0,1.0,2.0,1.0,3.0,...,NaN,NaN,NaN,NaN,3.0,NaN,1.0,1.0,0.0,1.0
293614,53,4.0,2,1,1.0,7.0,1.0,1.0,1.0,1.0,...,2.0,NaN,2.0,NaN,1.0,20.0,1.0,2.0,0.0,1.0


In [21]:
n_nan_cols = (df_all.isna().mean() <= 0.2).sum()
print(n_nan_cols)

147


#### Criando dicionarios

In [1]:
escolaridade_agregada_values = {
    1: 'Sem instrução e fundamental incompleto',
    2: 'Fundamental completo e médio incompleto',
    3: 'Médio completo e superior incompleto',
    4: 'Superior completo'
}
escolaridade_agregada_row = {
    'index': 'escolaridade_agregada',
    'origin': '',
    'desc': 'Escolaridade da população em novo padrão',
    'pv': escolaridade_agregada_values
}

In [2]:
desc_dict = {v: {'origin': k} for k, v in SELECTED_COLUMNS.items()}

desc_dict.update(UPDATE_DICT)

df_dict = pd.DataFrame.from_dict(PNS_DICT_RAW.complete_dictionary(desc_dict), orient='index').reset_index()
df_dict = df_dict.append(escolaridade_agregada_row, ignore_index = True)
a
df_dict.to_csv('../data/processed/PNS_2019/dictionary.csv', index=False)
df_dict

NameError: name 'SELECTED_COLUMNS' is not defined

Vamos embaralhar estes dados e salva-los na pasta processed/PNS/all.zip

In [23]:
df_all = df_all.convert_dtypes()
df_all = df_all.sample(frac=1, random_state=42).reset_index(drop=True)
df_all.to_csv('../data/processed/PNS_2019/all.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))

Dividindo as base em treino e teste

In [24]:
df_train, df_test = train_test_split(df_all, test_size=0.5, random_state=42)
df_train.to_csv('../data/processed/PNS_2019/train.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))
df_test.to_csv('../data/processed/PNS_2019/test.zip', index=False, compression=dict(method='zip', archive_name='data.csv', compresslevel=9))